# Geopandas

https://geopandas.org/ 

Est la bibliothèque bermettant d'utiliser des dataframe géographiques (geodataframes). Vous devirez déjà l'avoir installée mais dans le doute, remettons la procédure

## Bases 

### tutos

* https://www.youtube.com/watch?v=t7lliJXFt8w : introduction à geopandas
* https://www.youtube.com/watch?v=CtPqQP45vl0&list=PLewNEVDy7gq3DjrPDxGFLbHE4G2QWe8Qh&index=1 : les 7 premières vidéos

### geodataframe

In [ ]:
import geopandas as gpd

Importer des données se fait avec [read_file](https://geopandas.org/reference/geopandas.read_file.html?highlight=read_file). On l'a vu en 1.1 cette méthode lit tout un tas de format de données (geojson, shp, fichier zip shp, ...) qu'ils soient le local sur votre disque dur ou sur internet.


In [ ]:
métropole = gpd.read_file("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions.geojson")

In [ ]:
métropole.head()

geopandas manipule des [GeoDataframe](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html#geopandas.GeoDataFrame) tout comme pandas manipule des [Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). 

La principale différence visible est un type geometry (qui est dans la colonne `geometry`) qui contient nos données spaciales :

In [ ]:
métropole.dtypes

Et un crs associé à ces données : 

In [ ]:
métropole.crs

La colonne nommée geometry est une [GeoSeries](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.html). 

Il est **indispensable** qu'une geodataframe ait :

* une colonne nommée `geometry` qui soit une geoseries (une geoseries qui n'est pas nommée geometry ne sera pas prise en compte et une colonne geometry qui n'est pas une geoserie fera planter tôt ou tard geopandas).
* un crs de renseigné (sinon, tôt ou tard un calcul ne se fera pas comme il faut).

In [ ]:
métropole['geometry']

Le format du champ `geometry` est fixé (voir [la doc](https://fr.wikipedia.org/wiki/GeoJSON#Objets_g%C3%A9om%C3%A9triques)), et permet de gérer les objets primitifs suivant:
- des points `POINT`
- des lignes `LINESTRING`
- des polygones `POLYGON`

Ou des listes de même nature `MULTIPOINT`, `MULTILINESTRING` et `MULTIPOLYGON`.

> **Note** : il n'a pas de cercle car la projection de cercles sur des cartes peut être compliquée.

En python, ces formes sont gérées par la bibliothèque [shapely](https://shapely.readthedocs.io/en/latest/)

In [ ]:
type(métropole['geometry'][0])

On peut bien sur représenter graphiquement le dataframe :

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

métropole.plot(ax=ax)

plt.show()

Qui est équivalent à représenter juste sa géométrie : 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

métropole['geometry'].plot(ax=ax)

plt.show()

ou une forme (sans plot) :

In [ ]:
métropole['geometry'][0]

### Manipulations

Tou ce que vous faites avec pandas est également possible avec geopandas. On ajoute en plus des traitements de données spaciales 2D :

* des manipulations de données spaciales (grace à la bibliothèque [shapely](https://shapely.readthedocs.io/en/latest/))
* des représentation graphique avec [plot](https://geopandas.org/reference.html?highlight=plot#geopandas.GeoDataFrame.plot)

## Création d'un geodataframe

On va créer un geodataframe avec les villes française.

### Données brutes

> On va utiliser cet exemple poiur ontrer qu'il faut faire très attention lorsque l'on récupère des données sur internet. Tout le monde n'est pas soigneux dans le traitement de ses données...

Prenons les données de ce site : <https://sql.sh/736-base-donnees-villes-francaises> en csv.

In [ ]:
import pandas as pd

In [ ]:
villes_raw = pd.read_csv("https://sql.sh/ressources/sql-villes-france/villes_france.csv", 
                     header=None, index_col=0)

In [ ]:
villes_raw.dtypes

In [ ]:
villes_raw.head()

In [ ]:
villes_raw.loc[4440]

> **Attntion** aux données. L'analyse de Marseille montre que les noms de colonnes du site sont fantaisistes (ça n'a pas du passer les mise à jour successives...). Il vaut mieux mettre directement le nom des colonnes dans le tableau...

Après une analyse des correspondances on peut supposer que : 

- le nom : colonne 5
- la population en 2010 : colonne 14
- les coordonnées gps :
    - longitude : 19
    - latitude : 20

In [ ]:
mes_villes = villes_raw[[5, 14, 19, 20]].rename({5: "nom", 14: "population", 19: "longitude", 20: "latitude"}, axis=1)

In [ ]:
mes_villes

In [ ]:
mes_villes.loc[4440]

### Conversion en geodataframe

Pour obtenir une GeoDataFrame, il faut créer une colonne geometry. NOus allons utiliser les coordonnées GPS et donc associer un `POINT` pour chaque ligne. 

Il faut transformer nos coordonnées latitude/longitude GPS en point shapely.

#### A la main avec shapely

Nos coordonnées vont être des [Points](https://shapely.readthedocs.io/en/stable/manual.html#points), avec d'abord la longitude puis la latitude (abcisse puis ordonnée).

In [ ]:
from shapely.geometry import Point

In [ ]:
coordonnee_marseille = Point(mes_villes.loc[4440]['longitude'], mes_villes.loc[4440]['latitude'])

In [ ]:
print(coordonnee_marseille)

Il n'y a pas de crs par défaut sur un point, c'est juste un point. Pour transformer un point, il faut cependant connaitre son crs d'origne pour pouvoir trouver ses coordonnées dans le crs d'arrivé.

Le crs de shapely et de geopandas est géré par la bipliothèque [pyproj](http://pyproj4.github.io/pyproj/stable/). C'est très pratique car ce permet de faire toutes les projections que l'on veut très facilement (y'a juste un peu de doc à lire).

In [ ]:
import pyproj

Nos points sont dans le crs du wgs84, qui est le format lat/long couramment utilisé pour des coordonnées GPS:

In [ ]:
wgs84 = pyproj.CRS('epsg:4326') 

Essayons de convertir notre point dans un autre CRS. Par exemple dans les coordonnées [UTM](https://www.centcols.org/coord-utm/), qui permet de représenter des coordonnées geodésique dans le plan (voir [utiliser son gps](https://www.camptocamp.org/articles/427764/fr/-gps-en-pratique)).

Pour Marseille, c'est la 31ème zone de l'hémisphère nord donc l'epsg 32600 + 31 (32700 pour l'hémisphère sud) ce qui donne le CRS :

In [ ]:
utm = pyproj.CRS('epsg:32631')

Puis on convertit :

In [ ]:
from shapely.ops import transform

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

In [ ]:
utm_pt = transform(project, coordonnee_marseille)

In [ ]:
print(utm_pt)

Ouf, ça à l'air correct : [Zone 31T E: 692228.9 N: 4796466.46](https://www.gps-longitude-latitude.net/coordonnees-gps-de-marseille).


**Note** : l'epsg est vrai pour toute les latitude Nord, le [cadrillage UTM](https://fr.wikipedia.org/wiki/Syst%C3%A8me_de_r%C3%A9f%C3%A9rence_de_carroyage_militaire) est plus fin et sépare les latitude en 20 lettre (T, c'est dans le coin de Marseille).

#### Avec geopandas

On peut le faire directmeent en geopandas avec [points_from_xy](https://geopandas.org/reference/geopandas.points_from_xy.html) qui crée ainsi directement une colonne (notre geometry) : 

In [ ]:
mes_villes["longitude"]

In [ ]:
print(coordonnee_marseille)

In [ ]:
les_points = gpd.points_from_xy(x=mes_villes["longitude"], y=mes_villes["latitude"],
                   crs='epsg:4326')

In [ ]:
les_points

#### Création du geodataframe

On peut maintenant créer notre geodataframe

In [ ]:
villes = gpd.GeoDataFrame(mes_villes, geometry=les_points)

In [ ]:
villes.head()

In [ ]:
villes.crs

Y'a du monde dans le monde : 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

villes.plot(ax=ax)

plt.show()


Pour y voir plus clair utilisons la bibliothèque [`contextily`](https://contextily.readthedocs.io) pour ajouter un fond de carte :

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

villes.plot(ax=ax)
ctx.add_basemap(ax, crs="epsg:4326")

plt.show()

Y'a des choses bizarres... Mais avant de régler ces choses, utilisons le fond de carte de geoportail (`contextily` a [toute une liste](https://contextily.readthedocs.io/en/latest/providers_deepdive.html) de fond de cartes possible, n'hésitez pas à expérimenter) :

In [ ]:
import xyzservices.providers as xyz

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

villes.plot(ax=ax)
ctx.add_basemap(ax, crs="epsg:4326", source=xyz.GeoportailFrance.plan)

plt.show()

Revenons aux bizareries :

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

villes.plot(ax=ax)
ctx.add_basemap(ax, crs="epsg:4326")

ax.add_artist(Rectangle((-5, -50), 30, -15, 
                     facecolor="none", 
                     edgecolor="red"))
ax.text(-15, -55, "hein ?",
       color="red", size=20)

plt.show()

Il faudrait pouvoir connaitre les villes dans ce carré. Ca tombe bien c'est ce qu'on va apprendre tout de suite.

## Opérations sur des géodataframe

On ne va pas en faire beaucoup, je vous laisserai lire les docs (voir section tuto ci-après). Je vais juste essayer de vous mettre l'eau à la bouche.

### Point dans un polygone

Cherchons les villes dans le carré wtf.

In [ ]:
from shapely.geometry import Polygon

In [ ]:
sud = Polygon([(-5, -30), (-5, -45), (25, -45), (25, -30)])

In [ ]:
villes.geometry.within(sud)

In [ ]:
villes[villes.geometry.within(sud)]

Il n'y a personne...

C'est normal car les coordonnées wsg84 ne sont pas faitent pour être représentées sur un plan. La représentation graphique utilise la projection <https://epsg.io/3857>. Explicitons tout ça :

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15)) 

villes.to_crs("epsg:3857").plot(ax=ax)
ctx.add_basemap(ax, crs="epsg:3857")

plt.show()

La carte, donc la projection est bien identique. On peut maintenant faire notre intersection :

In [ ]:
villes[villes.to_crs("epsg:3857").geometry.within(Polygon([(-2*1e6, -5*1e6), (-2*1e6, -10*1e6), (5*1e6, -10*1e6), (5*1e6, -5*1e6)]))]

Parmi les 84 lignes présentent dans le caré, il y a  [POMPIDOU PAPA ICHTON](https://fr.wikipedia.org/wiki/Papaichton) est une ville de guyane.

Regardons cette villes dans le CRS pseudo-mercator de représentation graphique :

In [ ]:
villes.to_crs("epsg:3857").loc[36656]

Et dans le CRS wsg84 :

In [ ]:
villes.loc[36656]

Il semblerait que **Longitude et latitude ont été inversées pour certaines coordonnées**. Ce n'est pas sérieux pour des données disponible au téléchargement.

> sans une étude préalable des données ont aurait rien vu.

Comme il semble que pour la france métropolitaine les coordonnées sont exactes, restreignons nous à ces données.

### Associer sa région à une ville

<https://geopandas.org/en/stable/docs/user_guide/mergingdata.html>

In [ ]:
villes.head()

In [ ]:
métropole.head()

On va créer une nouvelle table avec pour chaque ville le nom de sa région. La jointure se fait sur les données spaciales :

In [ ]:
villes_métropole = gpd.sjoin(villes, métropole, how="inner", predicate='intersects')

In [ ]:
villes_métropole.head()

Il reste des colonnes en trop, épurons le tout :

In [ ]:
villes_métropole = villes_métropole.drop(columns=["index_right", "longitude", "latitude"]).rename({"nom_left": "nom", "nom_right": "région"}, axis=1)

In [ ]:
villes_métropole.head()

Réordonnons les colonnes : 

In [ ]:
villes_métropole = villes_métropole[['nom', 'région', 'population', 'geometry']]

In [ ]:
villes_métropole

### Compter le nombre de villes par régions

On va ajouter une colonne à régions qui contient le nombre de villes que possède la région.

#### nombre de villes par régions

On commence par compter le nombre de villes par région avec un [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).

1. On crée une colonne de 1 que l'on va sommer
2. On groupe par nom de région
3. on somme les colonnes groupées

On obtient une dataframe où tout ce qui est sommable l'a été, en particulier notre colonne de 1 :

In [ ]:
villes_métropole.assign(nombre=1)

In [ ]:
compte = (villes_métropole
             .assign(nombre=1)
             .groupby(by="région", as_index=False)
             .sum(numeric_only=True)
        )
compte

#### ajout de la colonne nombre pour les régions

Il nous reste à merger cette nouvelle table dans le geodataframe des régions :

1. Il faut que la colonne de merge ait le même nom donc on commence par renommer la colonne de compte
2. on peut ensuite faire le merge

On a utilisé l'attribut `inplace=True` qui modifie la dataframe plutôt que d'en créer une nouvelle. Utiliser ça avec parcimonie, car modifer une dataframe n'est presque jamais une bonne solution.

In [ ]:
compte.rename(columns={"région": "nom"}, inplace=True)

Tout est prêt on peut merger le résultat. On a pas besoin de toutes les colonnes de compte, juste de `'nom'` et `'nombre'` :

In [ ]:
villes_métropole

In [ ]:
métropole = métropole.merge(compte[['nom', 'nombre']], how='inner', on='nom')

In [ ]:
métropole

## Carte chloroplète

Une [carte chloroplète](https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the) associe donnée et cartographie.

### Graphique

La géographie peut se représenter directement avec GeoPandas :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.axis(False)

métropole.plot(ax=ax)

plt.show()

L'idée est de colorier chaque région en fonction du nombre. Ce si se fait facilement en ajoutant des paramètres (voir [cette doc](https://geopandas.org/en/stable/docs/user_guide/mapping.html)) :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.axis(False)

métropole.plot(column='nombre', 
               legend=True,
               legend_kwds={'label': "Nombre de villes", 'orientation': "horizontal"},
               cmap='OrRd',
               ax=ax)


plt.show()

On pourrait utiliser ce qu'on a vu précédemment pour colorier chaque région avec une coluleur spécifique et ainsi faire de plus jolies graphiques, mais plot permet d'avoir un résultat rapide et consitue une première solution.

### Cartes interractives

On utilise souvent une carte interactive pour représenter ces cartes chloroplète.  

Geopandas permet de le faire directeemnt avec la méthode `explore` (voir [la doc](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html)) :

In [ ]:
métropole.explore(column='nombre',
                  cmap='OrRd')

Geopandas utilise la bibliothèque javascript <https://leafletjs.com/> pour afficher ses cartes interractive dans jupyter. On peut également l'utiliser directement via la bibliothèque python [`folium`](https://python-visualization.github.io/folium/latest/).



Quelques tutos : 

* <https://geopandas.org/en/stable/gallery/plotting_with_folium.html>
* <https://python-visualization.github.io/folium/quickstart.html>
* ce qu'on va faire mais avec Paris :  <https://fxjollois.github.io/cours-2016-2017/analyse-donnees-massives-tp9.html>
* quelques exemple de cartes : <https://python-graph-gallery.com/288-map-background-with-folium/>

Plan :

1. carte centrée sur la France métropolitaine
2. ajout des régions
3. le nombre
4. on ajoute ecm avec un marker

#### Carte de France métropolitaine

On commence par trouver les coordonnées GPS du [centre de la France métropolitaine](https://fr.wikipedia.org/wiki/Centre_de_la_France) à Bruère-Allichamps (ce n'est pas la [gare de Perpignan](https://www.radiofrance.fr/franceculture/podcasts/l-esprit-des-lieux-la-chronique-de-l-ete/la-gare-de-perpignan-ou-salvador-dali-declara-le-centre-du-monde-8867048), qui est elle le centre du monde).

In [ ]:
from geopy import Nominatim # geodécodeur d'openstretmap

locator = Nominatim(user_agent="jupiter cours")

In [ ]:
centre_Fr = locator.geocode("Bruère-Allichamps, France")

centre_Fr

In [ ]:
centre_Fr.point[:2]

In [ ]:
import folium

In [ ]:
m = folium.Map(location=centre_Fr.point[:2],
               #tiles='Stamen Terrain',
               zoom_start=6
              )

In [ ]:
m

#### Ajout des régions

On peut utiliser explore pour ajouter un geodataframe à notre carte, mais par féfaut, folium ne connait que le format [Geojson](https://fr.wikipedia.org/wiki/GeoJSON), qui est un standard de gestion de données.

In [ ]:
data_geojson = métropole.to_json() # conversion en geojson (c'est du texte)

Le format geojson a une structure déterminé. C'est un dictionnaire de clés :

- `type` (nous c'est une featureCollection)
- `features` : liste des features

Chaque feature est un dictionnaire avec les clés :

- `id` : identifiant unique de la feature, nous des entiers
- `type` : nous une feature
- `properties` : les méta-données
- `geometry` : qui contient la géométrie de la feature. Nous ce sera des polygones déterminant las régions

Ce format est utilisé par folium. Voir : <https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html>

In [ ]:
import json 

# json.loads(data_geojson)  # affichage des propriétés

In [ ]:
json.loads(data_geojson)['features'][0]['properties']

In [ ]:
layer_regions = folium.GeoJson(
    data_geojson,
    name='geojson',
    tooltip=folium.features.GeoJsonTooltip(fields=['nom',]),  # sur le champ properties
)
layer_regions.add_to(m)

In [ ]:
m

#### Ajout du nombre

On va utiliser les données de la geodataframe pour réaliser la carte chloroplète. Il faut refaire la carte car on a ajouté les régions qui sont inutiles ici.

In [ ]:
layer_chloro = folium.Choropleth(
    geo_data=data_geojson,
    name='choropleth',
    data=métropole,
    columns=['code', 'nombre'],
    key_on='properties.code',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Nombre de pizzerias (%)'
)

layer_chloro.add_to(m)

In [ ]:
m

On a supperposé les deux layers sur la carte et la choloplète est au-dessus des régions, ce qui empêche le tooltip.

On va forcer l'ordre des deux layers sur la carte.

In [ ]:
m.keep_in_front(layer_chloro, layer_regions)

Dans l'idéal il faudrait supprimer la couleur bleue du layer région pour ne garder que les contours. Mais on va laisser ça en exercice, c'est facile avec la [doc](https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html#Styling) 

In [ ]:
m

Terminons cette partie en ajoutant un contrôle des layers :

In [ ]:
folium.LayerControl().add_to(m)

In [ ]:
m

#### Ajout d'une marque

Nous allons ajouter la marque de l'ecm sur la carte.

Commençons par trouver la localisation de l'ecm :

In [ ]:
from geopy import Nominatim
locator = Nominatim(user_agent="jupiter cours")
location = locator.geocode("ecole centrale de marseille, Marseille, France")

Les coordonnées sont :

In [ ]:
(location.latitude, location.longitude)

Et ajoutons une marque à la carte. On a tout de suite ajouté une [icône personalisée](https://python-visualization.github.io/folium/latest/user_guide/ui_elements/icons.html) et ajouté un [pop-up](https://python-visualization.github.io/folium/latest/user_guide/ui_elements/popups.html)

In [ ]:
folium.Marker(
    [location.latitude, location.longitude],
    icon=folium.Icon(color='beige', prefix="fa", icon='graduation-cap'),
    popup='ecm'
).add_to(m)

m

## Pour aller plus loin

### Opérations sur les géométries

Pour aller plus loin dans l'utilsation de shapely, vous pouvez :
* regarder ce tuto shapely en 2 parties : https://www.youtube.com/watch?v=LwpqA2WMR_8 et https://www.youtube.com/watch?v=3fm7x3TRcWs
* lire [la doc](https://shapely.readthedocs.io/en/latest/). Elle fourmille d'exemples sur les manipulations et conversions que l'on peut faire


### opérations sur les géodataframes

* https://www.youtube.com/watch?v=HtYxzt55-1w : les opérations geométriques disponibles très bien fait !)
* sur le [site de geopandas](https://geopandas.org/) même. Les user guide sont super bien. Par exemple :
    * https://geopandas.org/set_operations.html 
    * https://geopandas.org/geometric_manipulations.html

### Geoplot

Pour dessiner plus de choses. C'est une surcouche de cartopy donc l'installation est plus ardue.
<https://residentmario.github.io/geoplot/>